# TD Solutions — Business Intelligence (SQL avec SQLite)

## Résumé des TD

- **TD0** : Limites OLTP, nécessité DWH/OLAP
- **TD1** : Modèle en étoile (DDL, inserts, requêtes validation)
- **TD2** : Opérations OLAP (ROLLUP, fonctions fenêtre, contrôles)
- **TD3** : Choix architecture (ROLAP/MOLAP/HOLAP)

Ce notebook exécute les solutions SQL des TD1 et TD2 avec SQLite.

In [1]:
# Import SQLite et création base
import sqlite3
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
print('Base SQLite en mémoire créée')

Base SQLite en mémoire créée


## TD1 — Modèle en étoile

### DDL des tables

In [2]:
# DDL dimensions et faits
cursor.execute('''
CREATE TABLE dim_date (
  date_id INTEGER PRIMARY KEY AUTOINCREMENT,
  date_cal DATE NOT NULL,
  annee INTEGER,
  mois INTEGER,
  jour INTEGER,
  trimestre INTEGER
);
''')

cursor.execute('''
CREATE TABLE dim_produit (
  produit_sk INTEGER PRIMARY KEY AUTOINCREMENT,
  produit_id TEXT UNIQUE,
  produit_nom TEXT,
  categorie TEXT
);
''')

cursor.execute('''
CREATE TABLE dim_magasin (
  magasin_sk INTEGER PRIMARY KEY AUTOINCREMENT,
  magasin_id TEXT UNIQUE,
  ville TEXT,
  region TEXT
);
''')

cursor.execute('''
CREATE TABLE fact_ventes (
  fact_sk INTEGER PRIMARY KEY AUTOINCREMENT,
  date_id INTEGER NOT NULL,
  produit_sk INTEGER NOT NULL,
  magasin_sk INTEGER NOT NULL,
  montant REAL,
  quantite INTEGER,
  FOREIGN KEY (date_id) REFERENCES dim_date(date_id),
  FOREIGN KEY (produit_sk) REFERENCES dim_produit(produit_sk),
  FOREIGN KEY (magasin_sk) REFERENCES dim_magasin(magasin_sk)
);
''')
print('Tables créées')

Tables créées


### Inserts d'exemple

In [3]:
# Inserts dimensions
cursor.executemany('''
INSERT INTO dim_date (date_cal, annee, mois, jour, trimestre) VALUES (?, ?, ?, ?, ?);
''', [
    ('2024-01-02', 2024, 1, 2, 1),
    ('2024-01-03', 2024, 1, 3, 1),
    ('2024-01-04', 2024, 1, 4, 1)
])

cursor.executemany('''
INSERT INTO dim_produit (produit_id, produit_nom, categorie) VALUES (?, ?, ?);
''', [
    ('P01', 'Chemise Oxford', 'Textile'),
    ('P02', 'Sneakers Run', 'Chaussure'),
    ('P03', 'Jeans Slim', 'Textile')
])

cursor.executemany('''
INSERT INTO dim_magasin (magasin_id, ville, region) VALUES (?, ?, ?);
''', [
    ('M01', 'Paris', 'IDF'),
    ('M02', 'Lyon', 'ARA')
])

# Inserts faits
cursor.executemany('''
INSERT INTO fact_ventes (date_id, produit_sk, magasin_sk, montant, quantite) VALUES (?, ?, ?, ?, ?);
''', [
    (1, 1, 1, 120.0, 2),
    (1, 2, 1, 75.0, 1),
    (2, 1, 2, 60.0, 1),
    (3, 3, 1, 90.0, 1)
])
print('Données insérées')

Données insérées


### Requêtes de validation

In [4]:
# Volume total
cursor.execute('SELECT COUNT(*) AS nb_lignes FROM fact_ventes;')
print('Volume total :', cursor.fetchone()[0])

# Top 3 produits par CA
cursor.execute('''
SELECT p.produit_nom, SUM(f.montant) AS ca
FROM fact_ventes f
JOIN dim_produit p ON f.produit_sk = p.produit_sk
GROUP BY p.produit_nom
ORDER BY ca DESC
LIMIT 3;
''')
print('Top 3 produits :')
for row in cursor.fetchall():
    print(row)

# CA par ville et par mois
cursor.execute('''
SELECT d.annee, d.mois, m.ville, SUM(f.montant) AS ca
FROM fact_ventes f
JOIN dim_date d ON f.date_id = d.date_id
JOIN dim_magasin m ON f.magasin_sk = m.magasin_sk
GROUP BY d.annee, d.mois, m.ville
ORDER BY d.annee, d.mois, m.ville;
''')
print('CA par ville/mois :')
for row in cursor.fetchall():
    print(row)

Volume total : 4
Top 3 produits :
('Chemise Oxford', 180.0)
('Jeans Slim', 90.0)
('Sneakers Run', 75.0)
CA par ville/mois :
(2024, 1, 'Lyon', 60.0)
(2024, 1, 'Paris', 285.0)


## TD2 — Opérations OLAP

### Requêtes OLAP

In [5]:
# CA mensuel par produit et magasin (simplifié, SQLite sans ROLLUP avancé)
cursor.execute('''
SELECT strftime('%Y-%m-01', d.date_cal) AS mois, p.produit_nom, m.magasin_id, SUM(f.montant) AS ca_mensuel
FROM fact_ventes f
JOIN dim_date d ON f.date_id = d.date_id
JOIN dim_produit p ON f.produit_sk = p.produit_sk
JOIN dim_magasin m ON f.magasin_sk = m.magasin_sk
GROUP BY strftime('%Y-%m-01', d.date_cal), p.produit_nom, m.magasin_id
ORDER BY mois, ca_mensuel DESC;
''')
print('CA mensuel :')
for row in cursor.fetchall():
    print(row)

# Pivot simplifié (CA par produit et magasin)
cursor.execute('''
SELECT p.produit_nom,
       SUM(CASE WHEN m.magasin_id = 'M01' THEN f.montant ELSE 0 END) AS ca_M01,
       SUM(CASE WHEN m.magasin_id = 'M02' THEN f.montant ELSE 0 END) AS ca_M02
FROM fact_ventes f
JOIN dim_produit p ON f.produit_sk = p.produit_sk
JOIN dim_magasin m ON f.magasin_sk = m.magasin_sk
GROUP BY p.produit_nom;
''')
print('Pivot CA :')
for row in cursor.fetchall():
    print(row)

CA mensuel :
('2024-01-01', 'Chemise Oxford', 'M01', 120.0)
('2024-01-01', 'Jeans Slim', 'M01', 90.0)
('2024-01-01', 'Sneakers Run', 'M01', 75.0)
('2024-01-01', 'Chemise Oxford', 'M02', 60.0)
Pivot CA :
('Chemise Oxford', 120.0, 60.0)
('Jeans Slim', 90.0, 0)
('Sneakers Run', 75.0, 0)


### Contrôles de cohérence

In [6]:
# Total annuel = somme des mois
cursor.execute('''
SELECT annee, SUM(ca_mensuel) AS ca_annuel
FROM (
  SELECT strftime('%Y', d.date_cal) AS annee,
         strftime('%Y-%m-01', d.date_cal) AS mois,
         SUM(f.montant) AS ca_mensuel
  FROM fact_ventes f
  JOIN dim_date d ON f.date_id = d.date_id
  GROUP BY strftime('%Y', d.date_cal), strftime('%Y-%m-01', d.date_cal)
) x
GROUP BY annee;
''')
print('Total annuel :')
for row in cursor.fetchall():
    print(row)

# CA magasin = somme CA produits
cursor.execute('''
SELECT m.magasin_id, SUM(f.montant) AS ca_magasin
FROM fact_ventes f
JOIN dim_magasin m ON f.magasin_sk = m.magasin_sk
GROUP BY m.magasin_id;
''')
print('CA par magasin :')
for row in cursor.fetchall():
    print(row)

conn.close()
print('Notebook terminé')

Total annuel :
('2024', 345.0)
CA par magasin :
('M01', 285.0)
('M02', 60.0)
Notebook terminé
